In [44]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

In [ ]:
L = 1
T = 50
dt, dx = 1e-4, 1e-3

x = np.arange(0, L+dx, dx)
t = np.arange(0, T+dt, dt)

In [46]:
Nt, Nx = len(t), len(x)
Nt

500001

In [47]:

def u_0(x):
    return np.where((0.5-2*dx <= x) & (x <= 0.5+2*dx), 10, 0.0)

def v_0(x):
    "Initial Condition"
    return 0

def u_b(ti):
    return 0

c = 5e-2
U = np.zeros((2, Nt, Nx))

BC = "Derichlet"

def Du(ti, ui):
    du = np.zeros_like(ui)
    du[1:-1] = (ui[2:] - ui[:-2])/(2*dx)
    du[0]  = (ui[1]   - 0)/(dx)
    du[-1] = (u_b(ti) - ui[-2])/(dx)
    return du

def D2u(ti, ui):
    d2u       = np.zeros_like(ui)
    d2u[1:-1] = (ui[2:] - 2*ui[1:-1] + ui[:-2])/(dx**2)
    d2u[0]    = (u_b(ti) - 2*ui[1]  + ui[2])/(dx**2)
    d2u[-1]   = (u_b(ti) - 2*ui[-2] + ui[-3])/(dx**2)
    return d2u

def PDE(ti, Xi):
    dXi = np.array([Xi[1], c**2 * D2u(ti, Xi[0])])
    return dXi

for i, ti in enumerate(t):
    if i == 0:
        U[0, 0, :] = u_0(x)
        U[1, 0, :] = v_0(x)
    else:
        ui = U[0, i-1, :]
        vi = U[1, i-1, :]
        
        k1_u = dt*(vi)
        k1_v = dt*(c**2 * D2u(ti, ui))

        k2_u = dt*(vi + k1_u*0.5)
        k2_v = dt*(c**2 * D2u(ti + 0.5*dt, ui + 0.5*k1_v))

        k3_u = dt*(vi + k2_u*0.5)
        k3_v = dt*(c**2 * D2u(ti + 0.5*dt, ui + 0.5*k2_v))

        k4_u = dt*(vi + k3_u)
        k4_v = dt*(c**2 * D2u(ti + 0.5*dt, ui + k3_v))

        U[0, i, :] = ui + (1/6)*(k1_u + 2*k2_u + 2*k3_u + k4_u)
        U[1, i, :] = vi + (1/6)*(k1_v + 2*k2_v + 2*k3_v + k4_v)
        U[0, i, [0, -1]] = u_b(ti)

In [48]:
def save_gif_PIL(outfile, files, fps=5, loop=0):
    "Helper function for saving GIFs"
    imgs = [Image.open(file) for file in files]
    imgs[0].save(fp=outfile, format='GIF', append_images=imgs[1:], save_all=True, duration=int(1000/fps), loop=loop)

def make_plot(save=False, file_name=f"PDE_solution.png"):
    "Helper function for plotting"
    fig, ax = plt.subplots()
    ax.plot(x, U[0, i, :])
    ax.set_xlim(0, L)
    ax.set_ylim(-10.05, 10.05)
    ax.grid()
    ax.set_title(f"Time = {t[i]:.3f}s")
    if save:
        fig.savefig(file_name, bbox_inches="tight", pad_inches=0.1, dpi=100, facecolor="white")
        

In [49]:
import os
import shutil

folder_path="plots"
for item in os.listdir(folder_path):
    item_path = os.path.join(folder_path, item)
    try:
        if os.path.isfile(item_path) or os.path.islink(item_path):
            os.unlink(item_path)  # Remove file or symbolic link
        elif os.path.isdir(item_path):
            shutil.rmtree(item_path)  # Remove subdirectory and its contents
    except Exception as e:
        print(f"Failed to delete {item_path}. Reason: {e}")

In [50]:
files = []
for i in range(len(t)):
    if i%1000 == 0:
        file = f"plots/PDE_sol_{i:08d}.png"
        make_plot(save=True, file_name=file)
        files.append(file)
        plt.close("all")
save_gif_PIL("PDE_solution.gif", files, fps=20, loop=0)